In [237]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sy
from sympy import I as I
from sympy import pi as pi
import sympy.physics.quantum as qm
from sympy.physics.quantum import TensorProduct as tensor
from sympy.physics.quantum import OuterProduct as outer
%config InlineBackend.figure_format='retina'

In [287]:
# Quantum states
ket1 = qm.state.OrthogonalKet(1)
ket0 = qm.state.OrthogonalKet(0)
bra1 = qm.state.OrthogonalBra(1)
bra0 = qm.state.OrthogonalBra(0)
kets = np.array([ket1,ket0])
bras = np.array([bra1,bra0])

In [325]:
# Matrices
iden = np.array([[1,0],[0,1]])
sigx = np.array([[0,1],[1,0]])
sigy = np.array([[0,-I],[I,0]])
sigz = np.array([[1,0],[0,-1]])

def qop(A):
    retval = 0
    if type(A) != sy.matrices.immutable.ImmutableDenseMatrix:
        for ii,row in enumerate(A):
            for jj,val in enumerate(row):
                    retval += val* kets[ii]*bras[jj]
    else:
        for idx, val in enumerate(A):
            ii = int(idx/2)
            jj = int(idx - ii*np.sqrt(len(A)))
            retval += val*  kets[ii]*bras[jj]
    return retval
        

I_ = qop(iden)
X  = qop(sigx)
Y  = qop(sigy)
Z  = qop(sigz)

In [326]:
Z

-|0><0| + |1><1|

In [327]:
def Rx(theta):
    return sy.simplify(sy.exp(-I*theta/2 * sy.Matrix(sigx)))

def Ry(theta):
    return sy.simplify(sy.exp(-I*theta/2 * sy.Matrix(sigy)))

def Rz(phi):
    return sy.exp(-I*phi/2 * sy.Matrix(sigz))

def nhat(theta,phi): 
    return [sy.cos(phi)*sy.sin(theta), sy.sin(phi)*sy.sin(theta), sy.cos(theta)]

sigarrs = [sigx,sigy,sigz]
sigoper = [X,Y,Z]

def ndotsigma(theta,phi):
    rotax = nhat(theta,phi)
    retval = sy.Matrix.zeros(2)
    for comp,sig in zip(rotax,sigarrs):
        retval += comp*sig
    return retval

In [328]:
theta, phi = sy.symbols(r'\theta \phi', real=True)
# State with Bloch vector (r=1,theta,phi)
psi = qm.qapply(qop(Rz(phi)*Ry(theta))*ket1)

In [329]:
Rz(phi)*Ry(theta)

Matrix([
[exp(-I*\phi/2)*cos(\theta/2), -exp(-I*\phi/2)*sin(\theta/2)],
[ exp(I*\phi/2)*sin(\theta/2),   exp(I*\phi/2)*cos(\theta/2)]])

In [332]:
psi

exp(I*\phi/2)*sin(\theta/2)*|0> + exp(-I*\phi/2)*cos(\theta/2)*|1>

In [333]:
Xphi = Rz(phi)*sigx*Rz(-phi)

In [334]:
Xphi

Matrix([
[          0, exp(-I*\phi)],
[exp(I*\phi),            0]])

In [335]:
Rx(pi)

Matrix([
[ 0, -I],
[-I,  0]])

In [336]:
Ry(pi)

Matrix([
[0, -1],
[1,  0]])

In [337]:
sy.simplify(sy.exp(-I*pi/2*ndotsigma(sy.pi/2,phi)))

Matrix([
[             0,                            -sin(\phi) - I*cos(\phi)],
[-I*exp(I*\phi), exp(I*\phi)*sin(\phi) + I*exp(I*\phi)*cos(\phi) - I]])

In [338]:
sy.expand((-1)**(3/4)*(-1+sy.I)/2)

-5.55111512312578e-17 - 0.707106781186547*I

In [353]:
def gpi(phi):
    """
    @brief A pi rotation about the axis (theta, phi) = (pi/2, phi).
    
    @param phi Azimuthal angle of the rotation axis.
    @return Unitary 2x2 SymPy Matrix
    """
    return sy.Matrix([[0, sy.exp(-I*phi)],
                      [sy.exp(I*phi), 0]])

def gpi2(phi):
    """
    @brief A pi/2 rotation about the axis (theta, phi) = (pi/2, phi).

    @param phi Azimuthal angle of the rotation axis.
    @return Unitary 2x2 SymPy Matrix
    """
    return 1/sy.sqrt(2)*sy.Matrix([[1, -I*sy.exp(-I*phi)],
                                   [-I*sy.exp(I*phi), 1]])

In [350]:
gpi(phi)

Matrix([
[          0, exp(-I*\phi)],
[exp(I*\phi),            0]])

In [418]:
gpi2(phi)

Matrix([
[               sqrt(2)/2, -sqrt(2)*I*exp(-I*\phi)/2],
[-sqrt(2)*I*exp(I*\phi)/2,                 sqrt(2)/2]])

In [433]:
eps = np.finfo(float).eps
equalcond = 1
for phi_ in np.linspace(0,2*np.pi,1000):
    equalcond *= np.prod(np.abs(sy.N(gpi2(phi_)) - sy.N(sy.exp(-sy.I*sy.pi/4*ndotsigma(sy.pi/2,phi_)))) < 10*eps)
print(equalcond)

1
